<a href="https://colab.research.google.com/github/jonathanold/mordecai3/blob/main/geoparsing_with_mordecai3_jo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%capture
# Install dependencies

# !pip install elasticsearch==7.14.0
# !apt install default-jdk > /dev/null
# !pip install -q mordecai3
!pip install spacy==3.6.1
!python -m spacy download en_core_web_trf

2024-02-26 18:52:25.292285: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 18:52:25.292343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 18:52:25.293771: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 18:52:25.309236: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-26 18:52:27.011946: W tensorflow/compiler/tf2

In [7]:
# Download & extract Elasticsearch 7.13.0

!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.13.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.13.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.13.0

# Use prebuilt indexes
Faster, but will not stay up to date.

In [8]:
%%capture
# Download & extract es-geonames prebuilt indexes

!gdown https://drive.google.com/uc?id=1AVR9NKi0NYGT6jKS4csl4B_W2Xktb_Rr
!tar -xzvf geonames_index.tar.gz -C /content/elasticsearch-7.13.0/

In [5]:
%%bash --bg
sudo -H -u daemon elasticsearch-7.13.0/bin/elasticsearch

In [10]:
# Sleep for few seconds to let the instance start.
import time
time.sleep(30)

In [11]:
%%bash
echo "Change disk availability limits..."
curl -X PUT "localhost:9200/_cluster/settings" -H 'Content-Type: application/json' -d'
{
  "transient": {
    "cluster.routing.allocation.disk.watermark.low": "10gb",
    "cluster.routing.allocation.disk.watermark.high": "5gb",
    "cluster.routing.allocation.disk.watermark.flood_stage": "4gb",
    "cluster.info.update.interval": "1m"
  }
}
'


Change disk availability limits...
{"acknowledged":true,"persistent":{},"transient":{"cluster":{"routing":{"allocation":{"disk":{"watermark":{"low":"10gb","flood_stage":"4gb","high":"5gb"}}}},"info":{"update":{"interval":"1m"}}}}}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   452  100   195  100   257   1217   1605 --:--:-- --:--:-- --:--:--  2842


# Build ES-Geonames indexes
Very slow (approx. 40 min.), but latest indexes.

In [ ]:
%%capture
!pip install -q textacy

In [ ]:
%%bash --bg
sudo -H -u daemon elasticsearch-7.13.0/bin/elasticsearch

In [ ]:
# Sleep for few seconds to let the instance start.
import time
time.sleep(30)

In [ ]:
%%writefile geonames_mapping.json
{
    "settings" : {
        "number_of_shards" : 1,
        "number_of_replicas" : 1
    },
    "mappings" : {
        "properties" : {
          "geonameid" : {"type" : "keyword", "index": "true"},
          "name" : {"type" : "text"},
          "asciiname" : {"type" : "text"},
          "alternativenames" : {"type" : "text", "similarity" : "boolean",
                                "norms": false},
          "coordinates" : {"type" : "geo_point"},
          "feature_class" :  {"type" : "keyword", "index":  "true"},
          "feature_code" : {"type" : "keyword", "index":    "true"},
          "country_code3" : {"type" : "keyword", "index":   "true"},
          "admin1_code" : {"type" : "keyword", "index":  "true"},
          "admin1_name" : {"type" : "keyword", "index":  "true"},
          "admin2_code" : {"type" : "keyword", "index":  "true"},
          "admin2_name" : {"type" : "keyword", "index":  "true"},
          "admin3_code" : {"type" : "keyword", "index":  "true"},
          "admin4_code" : {"type" : "keyword", "index":  "true"},
          "population" : {"type" :  "long"},
          "alt_name_length": {"type": "long"},
          "modification_date" : {"type" : "date", "format": "date"}
          }
       }
}

In [ ]:
%%bash
echo "Starting Docker container and data volume..."
# create the directory first to avoid permission issues when Docker is running as root
mkdir $PWD/geonames_index/
docker run -d -p 127.0.0.1:9200:9200 -e "discovery.type=single-node" -v $PWD/geonames_index/:/usr/share/elasticsearch/data elasticsearch:7.10.1

echo "Downloading Geonames gazetteer..."
wget https://download.geonames.org/export/dump/allCountries.zip
wget https://download.geonames.org/export/dump/admin1CodesASCII.txt
wget https://download.geonames.org/export/dump/admin2Codes.txt
echo "Unpacking Geonames gazetteer..."
unzip allCountries.zip

echo "Creating mappings for the fields in the Geonames index..."
curl -XPUT 'localhost:9200/geonames' -H 'Content-Type: application/json' -d @geonames_mapping.json

echo "Change disk availability limits..."
curl -X PUT "localhost:9200/_cluster/settings" -H 'Content-Type: application/json' -d'
{
  "transient": {
    "cluster.routing.allocation.disk.watermark.low": "10gb",
    "cluster.routing.allocation.disk.watermark.high": "5gb",
    "cluster.routing.allocation.disk.watermark.flood_stage": "4gb",
    "cluster.info.update.interval": "1m"
  }
}
'


In [ ]:
from elasticsearch import Elasticsearch, helpers
import csv
import sys
from datetime import datetime
from tqdm import tqdm
import time
from textacy.preprocessing.remove import accents as remove_accents

#csv.field_size_limit(sys.maxsize)
csv.field_size_limit()
es = Elasticsearch(urls='http://localhost:9200/', timeout=60, max_retries=2)


def iso_convert(iso2c):
    """
    Takes a two character ISO country code and returns the corresponding 3
    character ISO country code.
    Parameters
    ----------
    iso2c: A two character ISO country code.
    Returns
    -------
    iso3c: A three character ISO country code.
    """

    iso_dict = {"AD":"AND", "AE":"ARE", "AF":"AFG", "AG":"ATG", "AI":"AIA",
                "AL":"ALB", "AM":"ARM", "AO":"AGO", "AQ":"ATA", "AR":"ARG",
                "AS":"ASM", "AT":"AUT", "AU":"AUS", "AW":"ABW", "AX":"ALA",
                "AZ":"AZE", "BA":"BIH", "BB":"BRB", "BD":"BGD", "BE":"BEL",
                "BF":"BFA", "BG":"BGR", "BH":"BHR", "BI":"BDI", "BJ":"BEN",
                "BL":"BLM", "BM":"BMU", "BN":"BRN", "BO":"BOL", "BQ":"BES",
                "BR":"BRA", "BS":"BHS", "BT":"BTN", "BV":"BVT", "BW":"BWA",
                "BY":"BLR", "BZ":"BLZ", "CA":"CAN", "CC":"CCK", "CD":"COD",
                "CF":"CAF", "CG":"COG", "CH":"CHE", "CI":"CIV", "CK":"COK",
                "CL":"CHL", "CM":"CMR", "CN":"CHN", "CO":"COL", "CR":"CRI",
                "CU":"CUB", "CV":"CPV", "CW":"CUW", "CX":"CXR", "CY":"CYP",
                "CZ":"CZE", "DE":"DEU", "DJ":"DJI", "DK":"DNK", "DM":"DMA",
                "DO":"DOM", "DZ":"DZA", "EC":"ECU", "EE":"EST", "EG":"EGY",
                "EH":"ESH", "ER":"ERI", "ES":"ESP", "ET":"ETH", "FI":"FIN",
                "FJ":"FJI", "FK":"FLK", "FM":"FSM", "FO":"FRO", "FR":"FRA",
                "GA":"GAB", "GB":"GBR", "GD":"GRD", "GE":"GEO", "GF":"GUF",
                "GG":"GGY", "GH":"GHA", "GI":"GIB", "GL":"GRL", "GM":"GMB",
                "GN":"GIN", "GP":"GLP", "GQ":"GNQ", "GR":"GRC", "GS":"SGS",
                "GT":"GTM", "GU":"GUM", "GW":"GNB", "GY":"GUY", "HK":"HKG",
                "HM":"HMD", "HN":"HND", "HR":"HRV", "HT":"HTI", "HU":"HUN",
                "ID":"IDN", "IE":"IRL", "IL":"ISR", "IM":"IMN", "IN":"IND",
                "IO":"IOT", "IQ":"IRQ", "IR":"IRN", "IS":"ISL", "IT":"ITA",
                "JE":"JEY", "JM":"JAM", "JO":"JOR", "JP":"JPN", "KE":"KEN",
                "KG":"KGZ", "KH":"KHM", "KI":"KIR", "KM":"COM", "KN":"KNA",
                "KP":"PRK", "KR":"KOR", "XK":"XKX", "KW":"KWT", "KY":"CYM",
                "KZ":"KAZ", "LA":"LAO", "LB":"LBN", "LC":"LCA", "LI":"LIE",
                "LK":"LKA", "LR":"LBR", "LS":"LSO", "LT":"LTU", "LU":"LUX",
                "LV":"LVA", "LY":"LBY", "MA":"MAR", "MC":"MCO", "MD":"MDA",
                "ME":"MNE", "MF":"MAF", "MG":"MDG", "MH":"MHL", "MK":"MKD",
                "ML":"MLI", "MM":"MMR", "MN":"MNG", "MO":"MAC", "MP":"MNP",
                "MQ":"MTQ", "MR":"MRT", "MS":"MSR", "MT":"MLT", "MU":"MUS",
                "MV":"MDV", "MW":"MWI", "MX":"MEX", "MY":"MYS", "MZ":"MOZ",
                "NA":"NAM", "NC":"NCL", "NE":"NER", "NF":"NFK", "NG":"NGA",
                "NI":"NIC", "NL":"NLD", "NO":"NOR", "NP":"NPL", "NR":"NRU",
                "NU":"NIU", "NZ":"NZL", "OM":"OMN", "PA":"PAN", "PE":"PER",
                "PF":"PYF", "PG":"PNG", "PH":"PHL", "PK":"PAK", "PL":"POL",
                "PM":"SPM", "PN":"PCN", "PR":"PRI", "PS":"PSE", "PT":"PRT",
                "PW":"PLW", "PY":"PRY", "QA":"QAT", "RE":"REU", "RO":"ROU",
                "RS":"SRB", "RU":"RUS", "RW":"RWA", "SA":"SAU", "SB":"SLB",
                "SC":"SYC", "SD":"SDN", "SS":"SSD", "SE":"SWE", "SG":"SGP",
                "SH":"SHN", "SI":"SVN", "SJ":"SJM", "SK":"SVK", "SL":"SLE",
                "SM":"SMR", "SN":"SEN", "SO":"SOM", "SR":"SUR", "ST":"STP",
                "SV":"SLV", "SX":"SXM", "SY":"SYR", "SZ":"SWZ", "TC":"TCA",
                "TD":"TCD", "TF":"ATF", "TG":"TGO", "TH":"THA", "TJ":"TJK",
                "TK":"TKL", "TL":"TLS", "TM":"TKM", "TN":"TUN", "TO":"TON",
                "TR":"TUR", "TT":"TTO", "TV":"TUV", "TW":"TWN", "TZ":"TZA",
                "UA":"UKR", "UG":"UGA", "UM":"UMI", "US":"USA", "UY":"URY",
                "UZ":"UZB", "VA":"VAT", "VC":"VCT", "VE":"VEN", "VG":"VGB",
                "VI":"VIR", "VN":"VNM", "VU":"VUT", "WF":"WLF", "WS":"WSM",
                "YE":"YEM", "YT":"MYT", "ZA":"ZAF", "ZM":"ZMB", "ZW":"ZWE",
                "CS":"SCG", "AN":"ANT"}

    try:
        iso3c = iso_dict[iso2c]
        return iso3c
    except KeyError:
        print('Bad code: ' + iso2c)
        iso3c = "NA"
        return iso3c


def read_adm1(fn="admin1CodesASCII.txt"):
    adm1_dict = {}
    with open(fn, 'rt', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            adm1_dict[row[0]] = row[1]
    return adm1_dict


def read_adm2(fn="admin2Codes.txt"):
    adm2_dict = {}
    with open(fn, 'rt', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            adm2_dict[row[0]] = row[1]
    return adm2_dict

def documents(reader, adm1_dict, adm2_dict, expand_ascii=True):
    """
    Load Geonames entries provided by the `reader` into Elasticsearch.

    If `expand_ascii` = True, any alternative names with accents will have an
    accent-stripped form included in the alternative names list along with the original form.
    For example, the name "Ḩadīqat ash Shahbā" would generate a second entry
    "Hadiqat ash Shahba". This process does not affect non-Roman characters
    (e.g. "北京" remains "北京".)

    Parameters
    ----------
    reader: a CSV reader object
    adm1_dict: dict
      map from numeric ADM1 codes to names
    adm2_dict: dict
      map from numeric ADM2 codes to names
    expand_ascii: bool
      Include versions of names with accents stripped out.
    """
    todays_date = datetime.today().strftime("%Y-%m-%d")
    count = 0
    for row in tqdm(reader, total=12237435): # approx
        try:
            coords = row[4] + "," + row[5]
            country_code3 = iso_convert(row[8])
            alt_names = list(set(row[3].split(",")))
            # so annoying...add "US" as an alt name for USA
            if str(row[0]) == "6252001":
                alt_names.append("US")
                alt_names.append("U.S.")
            if str(row[0]) == "239880":
                alt_names.append("C.A.R.")
            alt_name_length = len(alt_names)
            if expand_ascii:
                stripped = [remove_accents(i) for i in alt_names]
                both = alt_names + stripped
                both = list(set(both))
                alt_names = both
            # get ADM1 name
            if row[10]:
                country_admin1 = '.'.join([row[8], row[10]])
                try:
                    admin1_name = adm1_dict[country_admin1]
                except KeyError:
                    admin1_name = ""
            else:
                admin1_name = ""
            # Get ADM2 name
            if row[11]:
                country_admin2 = '.'.join([row[8], row[10], row[11]])
                try:
                    admin2_name = adm2_dict[country_admin2]
                except KeyError:
                    admin2_name = ""
                    count += 1
            else:
                admin2_name = ""
            doc = {"geonameid" : row[0],
                    "name" : row[1],
                    "asciiname" : row[2],
                    "alternativenames" : alt_names,
                    "coordinates" : coords,  # 4, 5
                    "feature_class" : row[6],
                    "feature_code" : row[7],
                    "country_code3" : country_code3,
                    "admin1_code" : row[10],
                    "admin1_name": admin1_name,
                    "admin2_code" : row[11],
                    "admin2_name": admin2_name,
                    "admin3_code" : row[12],
                    "admin4_code" : row[13],
                    "population" : row[14],
                    "alt_name_length": alt_name_length,
                    "modification_date" : todays_date
                   }
            action = {"_index" : "geonames",
                      "_id" : doc['geonameid'],
                      "_source" : doc}
            yield action
        except Exception as e:
            print(e, row)
            count += 1
    print('Exception count:', count)



if __name__ == "__main__":
    t = time.time()
    adm1_dict = read_adm1()
    adm2_dict = read_adm2()
    f = open('allCountries.txt', 'rt', encoding='utf-8')
    reader = csv.reader(f, delimiter='\t')
    actions = documents(reader, adm1_dict, adm2_dict)
    helpers.bulk(es, actions, chunk_size=500)
    es.indices.refresh(index='geonames')
    e = (time.time() - t) / 60
    print("Elapsed minutes: ", e)


# Geoparser inference

In [2]:
!pip install --upgrade transformers==4.30.2
from mordecai3 import Geoparser
geo = Geoparser()

In [16]:
import pandas as pd

listx = ["A group of farmers rioted in Bhiwani, Haryana, the Times of Mumbai reported on Wednesday.", "Schwäbisch Hall is a beautiful town in Baden-Württemberg."]
combined_df = pd.DataFrame()

for z in range(0,len(listx)):
  b = geo.geoparse_doc(listx[z])
  # Extract the 'geolocated_ents' list and normalize it
  geolocated_ents_data = b.get('geolocated_ents', [])
  df = pd.json_normalize(geolocated_ents_data)

  # Combine the flattened data with the non-nested columns
  df = pd.concat([pd.DataFrame(b).drop('geolocated_ents', axis=1), df], axis=1)
  combined_df = combined_df.append(df, ignore_index=True)



b

/usr/local/lib/python3.10/dist-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
INFO:elasticsearch:POST http://localhost:9200/geonames/_search [status:200 request:0.054s]
2024-02-26 21:00:40,666 elasticsearch INFO     POST http://localhost:9200/geonames/_search [status:200 request:0.054s]
<ipython-input-16-07c326d2e514>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
/usr/local/lib/python3.10/dist-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without au

{'doc_text': 'Schwäbisch Hall is a beautiful town in Baden-Württemberg.',
 'event_location_raw': '',
 'geolocated_ents': [{'feature_code': 'ADM4',
   'feature_class': 'A',
   'country_code3': 'DEU',
   'lat': 49.11124,
   'lon': 9.73947,
   'name': 'Schwäbisch Hall',
   'admin1_code': '01',
   'admin1_name': 'Baden-Wurttemberg',
   'admin2_code': '081',
   'admin2_name': 'Regierungsbezirk Stuttgart',
   'geonameid': '6558007',
   'admin1_parent_match': 0,
   'country_code_parent_match': 0,
   'alt_name_length': 2.0794415416798357,
   'min_dist': 0.0,
   'max_dist': 0.0,
   'avg_dist': 0.0,
   'ascii_dist': 0.0,
   'adm1_count': 1.0,
   'country_count': 1.0,
   'score': 0.7057328820228577,
   'search_name': 'Schwäbisch Hall',
   'start_char': 0,
   'end_char': 26,
   'city_id': '',
   'city_name': ''},
  {'feature_code': 'ADM1',
   'feature_class': 'A',
   'country_code3': 'DEU',
   'lat': 48.5,
   'lon': 9.0,
   'name': 'Baden-Württemberg',
   'admin1_code': '01',
   'admin1_name': 'Ba

In [17]:

combined_df.to_csv("test.csv", index=False)
from google.colab import files
files.download("test.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>